In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model_name = 'Kimata/llama-2-7b-Vizuosense'


# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.float16
#     offload_folder = '/content/ckpt'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Load LLaMA tokenizer[/INST]
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


In [5]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = """[{'role': 'system',
  'content': "\n**Autonomous Driving Planner**\nRole: You are the brain of an autonomous vehicle. Plan a safe 3-second driving trajectory. Avoid collisions with other objects.\n\nContext\n- Coordinates: X-axis is perpendicular, and Y-axis is parallel to the direction you're facing. You're at point (0,0).\n- Objective: Create a 3-second route using 6 waypoints, one every 0.5 seconds.\n\nInputs\n1. Perception & Prediction: Info about surrounding objects and their predicted movements.\n2. Historical Trajectory: Your past 2-second route, given by 4 waypoints.\n3. Ego-States: Your current state including velocity, heading angular velocity, can bus data, heading speed, and steering signal.\n4. Mission Goal: Goal location for the next 3 seconds.\n\nTask\n- Thought Process: Note down critical objects and potential effects from your perceptions and predictions.\n- Action Plan: Detail your meta-actions based on your analysis.\n- Trajectory Planning: Develop a safe and feasible 3-second route using 6 new waypoints.\n\nOutput\n- Thoughts:\n  - Notable Objects\n    Potential Effects\n- Meta Action\n- Trajectory (MOST IMPORTANT):\n  - [(x1,y1), (x2,y2), ... , (x6,y6)]\n"},
 {'role': 'user',
  'content': '\nPerception and Prediction:\n - trafficcone at (5.25,0.04), moving to (5.25,0.01).\n - trafficcone at (-13.47,5.44), moving to (-13.47,5.40).\n - trafficcone at (-9.13,19.80), moving to (-9.12,19.76).\n - trafficcone at (-9.30,16.88), moving to (-9.30,16.88).\n - trafficcone at (-13.37,8.49), moving to (-13.36,8.41).\n - trafficcone at (-9.70,10.84), moving to (-9.71,10.79).\n - car at (1.08,15.43), moving to (1.16,16.90).\n - car at (0.50,9.57), moving to (0.52,9.56).\n - trafficcone at (-9.47,18.30), moving to (-9.47,18.30).\n - barrier at (-9.40,19.11), moving to (-9.43,18.96).\n - trafficcone at (5.47,3.14), moving to (5.46,3.14).\n - construction at (-11.76,14.44), moving to (-11.76,14.43).\n - trafficcone at (-9.29,13.81), moving to (-9.33,13.75).\n - truck at (3.59,7.69), moving to (3.59,7.79).\n - trafficcone at (-12.32,9.92), moving to (-12.32,9.92).\nEgo-States:\n - Velocity (vx,vy): (-0.00,0.00)\n - Heading Angular Velocity (v_yaw): (-0.00)\n - Acceleration (ax,ay): (0.00,0.00)\n - Can Bus: (-0.03,0.06)\n - Heading Speed: (0.00)\n - Steering: (-0.28)\nHistorical Trajectory (last 2 seconds): [(0.00,0.00), (0.00,0.00), (0.00,0.00), (0.00,0.00)]\nMission Goal: FORWARD\n'},
 {'role': 'assistant',
  'content': 'Thoughts:\n"""




pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=3000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] [{'role': 'system',
  'content': "
**Autonomous Driving Planner**
Role: You are the brain of an autonomous vehicle. Plan a safe 3-second driving trajectory. Avoid collisions with other objects.

Context
- Coordinates: X-axis is perpendicular, and Y-axis is parallel to the direction you're facing. You're at point (0,0).
- Objective: Create a 3-second route using 6 waypoints, one every 0.5 seconds.

Inputs
1. Perception & Prediction: Info about surrounding objects and their predicted movements.
2. Historical Trajectory: Your past 2-second route, given by 4 waypoints.
3. Ego-States: Your current state including velocity, heading angular velocity, can bus data, heading speed, and steering signal.
4. Mission Goal: Goal location for the next 3 seconds.

Task
- Thought Process: Note down critical objects and potential effects from your perceptions and predictions.
- Action Plan: Detail your meta-actions based on your analysis.
- Trajectory Planning: Develop a safe and feasible 3-sec

In [6]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = """Hello there."""




pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Hello there. [/INST]  Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [10]:
prompt = """
[{'role': 'system',
  'content': "\n**Autonomous Driving Planner**\nRole: You are the brain of an autonomous vehicle. Plan a safe 3-second driving trajectory. Avoid collisions with other objects.\n\nContext\n- Coordinates: X-axis is perpendicular, and Y-axis is parallel to the direction you're facing. You're at point (0,0).\n- Objective: Create a 3-second route using 6 waypoints, one every 0.5 seconds.\n\nInputs\n1. Perception & Prediction: Info about surrounding objects and their predicted movements.\n2. Historical Trajectory: Your past 2-second route, given by 4 waypoints.\n3. Ego-States: Your current state including velocity, heading angular velocity, can bus data, heading speed, and steering signal.\n4. Mission Goal: Goal location for the next 3 seconds.\n\nTask\n- Thought Process: Note down critical objects and potential effects from your perceptions and predictions.\n- Action Plan: Detail your meta-actions based on your analysis.\n- Trajectory Planning: Develop a safe and feasible 3-second route using 6 new waypoints.\n\nOutput\n- Thoughts:\n  - Notable Objects\n    Potential Effects\n- Meta Action\n- Trajectory (MOST IMPORTANT):\n  - [(x1,y1), (x2,y2), ... , (x6,y6)]\n"},
 {'role': 'user',
  'content': '\nPerception and Prediction:\n - trafficcone at (4.83,6.20), moving to (4.85,6.19).\n
  """

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=len(prompt) + 300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 
[{'role': 'system',
  'content': "
**Autonomous Driving Planner**
Role: You are the brain of an autonomous vehicle. Plan a safe 3-second driving trajectory. Avoid collisions with other objects.

Context
- Coordinates: X-axis is perpendicular, and Y-axis is parallel to the direction you're facing. You're at point (0,0).
- Objective: Create a 3-second route using 6 waypoints, one every 0.5 seconds.

Inputs
1. Perception & Prediction: Info about surrounding objects and their predicted movements.
2. Historical Trajectory: Your past 2-second route, given by 4 waypoints.
3. Ego-States: Your current state including velocity, heading angular velocity, can bus data, heading speed, and steering signal.
4. Mission Goal: Goal location for the next 3 seconds.

Task
- Thought Process: Note down critical objects and potential effects from your perceptions and predictions.
- Action Plan: Detail your meta-actions based on your analysis.
- Trajectory Planning: Develop a safe and feasible 3-se

In [11]:
prompt = """
[{'role': 'system',
  'content': "\n**Autonomous Driving Planner**\nRole: You are the brain of an autonomous vehicle. Plan a safe 3-second driving trajectory. Avoid collisions with other objects.\n\nContext\n- Coordinates: X-axis is perpendicular, and Y-axis is parallel to the direction you're facing. You're at point (0,0).\n- Objective: Create a 3-second route using 6 waypoints, one every 0.5 seconds.\n\nInputs\n1. Perception & Prediction: Info about surrounding objects and their predicted movements.\n2. Historical Trajectory: Your past 2-second route, given by 4 waypoints.\n3. Ego-States: Your current state including velocity, heading angular velocity, can bus data, heading speed, and steering signal.\n4. Mission Goal: Goal location for the next 3 seconds.\n\nTask\n- Thought Process: Note down critical objects and potential effects from your perceptions and predictions.\n- Action Plan: Detail your meta-actions based on your analysis.\n- Trajectory Planning: Develop a safe and feasible 3-second route using 6 new waypoints.\n\nOutput\n- Thoughts:\n  - Notable Objects\n    Potential Effects\n- Meta Action\n- Trajectory (MOST IMPORTANT):\n  - [(x1,y1), (x2,y2), ... , (x6,y6)]\n"},
 {'role': 'user',
  'content': '\nPerception and Prediction:\n - adult at (9.26,0.39), moving to (9.26,0.39).\n - bicycle at (-15.92,12.70), moving to unknown location.\n - truck at (-10.31,4.46), moving to (-7.89,-17.56).\n - car at (-15.47,18.41), moving to (-13.37,-1.99).\n - adult at (-19.45,3.48), moving to (-19.68,8.30).\n - adult at (-18.66,-2.51), moving to (-19.42,2.35).\n - car at (-11.52,15.16), moving to (-9.58,-3.13).\nEgo-States:\n - Velocity (vx,vy): (-0.02,4.35)\n - Heading Angular Velocity (v_yaw): (-0.00)\n - Acceleration (ax,ay): (0.10,-0.36)\n - Can Bus: (0.66,-0.02)\n - Heading Speed: (4.37)\n - Steering: (-0.10)\nHistorical Trajectory (last 2 seconds): [(0.09,-17.28), (0.10,-13.26), (0.05,-9.09), (-0.03,-4.37)]\nMission Goal: FORWARD\n'},
 {'role': 'assistant',
 """

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=len(prompt) + 300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 
[{'role': 'system',
  'content': "
**Autonomous Driving Planner**
Role: You are the brain of an autonomous vehicle. Plan a safe 3-second driving trajectory. Avoid collisions with other objects.

Context
- Coordinates: X-axis is perpendicular, and Y-axis is parallel to the direction you're facing. You're at point (0,0).
- Objective: Create a 3-second route using 6 waypoints, one every 0.5 seconds.

Inputs
1. Perception & Prediction: Info about surrounding objects and their predicted movements.
2. Historical Trajectory: Your past 2-second route, given by 4 waypoints.
3. Ego-States: Your current state including velocity, heading angular velocity, can bus data, heading speed, and steering signal.
4. Mission Goal: Goal location for the next 3 seconds.

Task
- Thought Process: Note down critical objects and potential effects from your perceptions and predictions.
- Action Plan: Detail your meta-actions based on your analysis.
- Trajectory Planning: Develop a safe and feasible 3-se